Copyright (c) 2023 Giovanni Squillero <giovanni.squillero@polito.it>
https://github.com/squillero/computational-intelligence
Free for personal or classroom use; see LICENSE.md for details.

In [ ]:
from itertools import product
from random import random, randint, choice, seed, choices
import numpy as np
from scipy import sparse
from copy import  copy
from functools import reduce

In [ ]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

# Halloween Challenge
Find the best solution with the fewest calls to the fitness functions for:

- num_points = [100, 1_000, 5_000]
- num_sets = num_points
- density = [.3, .7]

In [ ]:
x = make_set_covering_problem(1000, 1000, .3)
print("Element at row=42 and column=42:", x[42, 42])

In [ ]:
def fitness1(sets, state):
    cost = sum(state)
    valid = np.all(
        reduce(
            np.logical_or,
            [sets[i, :] for i, t in enumerate(state) if t],
            np.array([False for _ in range(sets.shape[1])]),
        )
    )
    return valid, -cost

def fitness2(sets, state):
    cost = sum(state)
    if np.array(state).any():
        valid = sets[np.array(state), :].max(axis=0).sum()
    else:
        valid = 0
    return valid, -cost

In [ ]:
fitness = fitness1

In [ ]:
def tweak(state, size):
    new_state = copy(state)
    index = randint(0, size - 1)
    new_state[index] = not new_state[index]
    return new_state


# Hill Climbing

In [ ]:
NUM_SETS = 100
sets = make_set_covering_problem(NUM_SETS, NUM_SETS, .3).toarray()
current_state = [choice([False, False, False, False, False, False]) for _ in range(NUM_SETS)]
print(fitness(sets, current_state))

for step in range(1000):
    new_state = tweak(current_state, NUM_SETS)
    if fitness(sets, new_state) >= fitness(sets, current_state):
        current_state = new_state
        print(fitness(sets, current_state))


# Simulated Annealinig

In [ ]:
def f(val):
    return val[0] * val[1]

NUM_SETS = 1000
sets = make_set_covering_problem(NUM_SETS, NUM_SETS, .3).toarray()
current_state = [choice([False, False, False, False, False, False]) for _ in range(NUM_SETS)]
print(fitness(sets, current_state))
schedule = 2
counter = 0
t = 15
for step in range(10_000):
    if counter % schedule == 0 and t > 1:
        t -= 1
    new_state = tweak(current_state, NUM_SETS)
    s = fitness(sets, current_state)
    s_new = fitness(sets, new_state)
    p = np.exp(-(f(s) - f(s_new)) / t)
    if s_new >= s or choices([False, True], weights=(1 - p, p)):
        current_state = new_state
        print(fitness(sets, current_state))
    counter += 1
